In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
from nmt_utils import *
import matplotlib.pyplot as plt
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
%matplotlib inline

Using TensorFlow backend.


In [2]:
m = 10000
dataset, human_vocab, machine_vocab, inv_machine_vocab = load_dataset(m)

100%|█████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 28563.46it/s]


In [3]:
dataset[:10]

[('9 may 1998', '1998-05-09'),
 ('10.09.70', '1970-09-10'),
 ('4/28/90', '1990-04-28'),
 ('thursday january 26 1995', '1995-01-26'),
 ('monday march 7 1983', '1983-03-07'),
 ('sunday may 22 1988', '1988-05-22'),
 ('tuesday july 8 2008', '2008-07-08'),
 ('08 sep 1999', '1999-09-08'),
 ('1 jan 1981', '1981-01-01'),
 ('monday may 22 1995', '1995-05-22')]

In [4]:
Tx = 30
Ty = 10
X, Y, Xoh, Yoh = preprocess_data(dataset, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (10000, 30)
Y.shape: (10000, 10)
Xoh.shape: (10000, 30, 37)
Yoh.shape: (10000, 10, 11)


In [5]:
index = 0
print("Source date:", dataset[index][0])
print("Target date:", dataset[index][1])
print()
print("Source after preprocessing (indices):", X[index])
print("Target after preprocessing (indices):", Y[index])
print()
print("Source after preprocessing (one-hot):", Xoh[index])
print("Target after preprocessing (one-hot):", Yoh[index])

Source date: 9 may 1998
Target date: 1998-05-09

Source after preprocessing (indices): [12  0 24 13 34  0  4 12 12 11 36 36 36 36 36 36 36 36 36 36 36 36 36 36
 36 36 36 36 36 36]
Target after preprocessing (indices): [ 2 10 10  9  0  1  6  0  1 10]

Source after preprocessing (one-hot): [[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Target after preprocessing (one-hot): [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [6]:
# Defined shared layers as global variables
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [7]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a"
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis
    concat = concatenator([a,s_prev])
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e.
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies.
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas"
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell
    context = dotor([alphas,a])
    
    return context

In [8]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation=softmax)

In [9]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
        
    # Define your pre-attention Bi-LSTM. Remember to use return_sequences=True.
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    # Iterate for Ty steps
    for t in range(Ty):
    
        # Perform one step of the attention mechanism to get back the context vector at step t 
        context = one_step_attention(a,s)
        
        # Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state]
        s, _, c = post_activation_LSTM_cell(context, initial_state = [s, c])
        
        # Apply Dense layer to the hidden state output of the post-attention LSTM
        out = output_layer(s)
        
        # Append "out" to the "outputs" list
        outputs.append(out)
    
    # Create model instance taking three inputs and returning the list of outputs.
    model = Model([X,s0,c0],outputs)
    
    return model

In [10]:
model = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 37)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 64)       17920       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 30, 64)       0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

                                                                 attention_weights[4][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[5][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[6][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[7][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[8][0]          
                                                                 bidirectional_1[0][0]            
          

In [12]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.001)
model.compile(opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
s0 = np.zeros((m, n_s))
c0 = np.zeros((m, n_s))
print(Yoh.shape)
outputs = list(Yoh.swapaxes(0,1))
print(Yoh.shape)

(10000, 10, 11)
(10000, 10, 11)


Let's now fit the model 

In [14]:
model.fit([Xoh, s0, c0], outputs, epochs=40, batch_size=100)

Epoch 1/40


 2800/10000 [=======>......................] - ETA: 13:59 - loss: 23.9800 - dense_3_loss: 2.3999 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.0000e+00 - dense_3_acc_2: 0.0000e+00 - dense_3_acc_3: 0.0900 - dense_3_acc_4: 0.0000e+00 - dense_3_acc_5: 0.0000e+00 - dense_3_acc_6: 0.0800 - dense_3_acc_7: 0.0000e+00 - dense_3_acc_8: 0.0000e+00 - dense_3_acc_9: 0.07 - ETA: 7:00 - loss: 23.8149 - dense_3_loss: 2.4035 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.2100 - dense_3_acc_2: 0.1150 - dense_3_acc_3: 0.0950 - dense_3_acc_4: 0.0100 - dense_3_acc_5: 0.3600 - dense_3_acc_6: 0.0750 - dense_3_acc_7: 0.0200 - dense_3_acc_8: 0.1750 - dense_3_acc_9: 0.0700                         - ETA: 4:41 - loss: 23.6351 - dense_3_loss: 2.4148 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.2700 - dense_3_acc_2: 0.1433 - dense_3_acc_3: 0.1033 - dense_3_acc_4: 0.0067 - dense_3_acc_5: 0.4967 - dense_3_acc_6: 0.0667 - dense_3_acc_7: 0.0133 - dense_3_acc_8: 0.1933 - dense_3_acc_9: 0.080 - ETA: 3:31 - loss: 23.3838 - 

 5700/10000 [================>.............] - ETA: 28s - loss: 21.1126 - dense_3_loss: 2.8060 - dense_3_acc: 0.0000e+00 - dense_3_acc_1: 0.4510 - dense_3_acc_2: 0.2041 - dense_3_acc_3: 0.0707 - dense_3_acc_4: 0.6859 - dense_3_acc_5: 0.1110 - dense_3_acc_6: 0.0152 - dense_3_acc_7: 0.8290 - dense_3_acc_8: 0.0614 - dense_3_acc_9: 0.03 - ETA: 26s - loss: 21.0445 - dense_3_loss: 2.8090 - dense_3_acc: 0.0097 - dense_3_acc_1: 0.4573 - dense_3_acc_2: 0.2107 - dense_3_acc_3: 0.0683 - dense_3_acc_4: 0.6963 - dense_3_acc_5: 0.1073 - dense_3_acc_6: 0.0147 - dense_3_acc_7: 0.8347 - dense_3_acc_8: 0.0617 - dense_3_acc_9: 0.0333   - ETA: 25s - loss: 20.9922 - dense_3_loss: 2.8154 - dense_3_acc: 0.0277 - dense_3_acc_1: 0.4610 - dense_3_acc_2: 0.2142 - dense_3_acc_3: 0.0661 - dense_3_acc_4: 0.7061 - dense_3_acc_5: 0.1039 - dense_3_acc_6: 0.0142 - dense_3_acc_7: 0.8400 - dense_3_acc_8: 0.0665 - dense_3_acc_9: 0.03 - ETA: 25s - loss: 20.9337 - dense_3_loss: 2.8158 - dense_3_acc: 0.0447 - dense_3_acc_1: 

 8600/10000 [========================>.....] - ETA: 10s - loss: 19.0701 - dense_3_loss: 2.7599 - dense_3_acc: 0.3426 - dense_3_acc_1: 0.6378 - dense_3_acc_2: 0.2893 - dense_3_acc_3: 0.0516 - dense_3_acc_4: 0.8369 - dense_3_acc_5: 0.1650 - dense_3_acc_6: 0.0159 - dense_3_acc_7: 0.8652 - dense_3_acc_8: 0.1279 - dense_3_acc_9: 0.07 - ETA: 10s - loss: 18.9709 - dense_3_loss: 2.7566 - dense_3_acc: 0.3500 - dense_3_acc_1: 0.6431 - dense_3_acc_2: 0.2910 - dense_3_acc_3: 0.0508 - dense_3_acc_4: 0.8397 - dense_3_acc_5: 0.1751 - dense_3_acc_6: 0.0163 - dense_3_acc_7: 0.8675 - dense_3_acc_8: 0.1332 - dense_3_acc_9: 0.07 - ETA: 9s - loss: 18.8729 - dense_3_loss: 2.7515 - dense_3_acc: 0.3595 - dense_3_acc_1: 0.6465 - dense_3_acc_2: 0.2917 - dense_3_acc_3: 0.0507 - dense_3_acc_4: 0.8423 - dense_3_acc_5: 0.1853 - dense_3_acc_6: 0.0170 - dense_3_acc_7: 0.8697 - dense_3_acc_8: 0.1375 - dense_3_acc_9: 0.0755 - ETA: 9s - loss: 18.7640 - dense_3_loss: 2.7456 - dense_3_acc: 0.3687 - dense_3_acc_1: 0.6513 -

10000/10000 [==============================] - ETA: 2s - loss: 16.3273 - dense_3_loss: 2.6175 - dense_3_acc: 0.5357 - dense_3_acc_1: 0.7431 - dense_3_acc_2: 0.3372 - dense_3_acc_3: 0.0700 - dense_3_acc_4: 0.8913 - dense_3_acc_5: 0.3824 - dense_3_acc_6: 0.0676 - dense_3_acc_7: 0.9101 - dense_3_acc_8: 0.2226 - dense_3_acc_9: 0.091 - ETA: 2s - loss: 16.2521 - dense_3_loss: 2.6141 - dense_3_acc: 0.5407 - dense_3_acc_1: 0.7457 - dense_3_acc_2: 0.3387 - dense_3_acc_3: 0.0706 - dense_3_acc_4: 0.8925 - dense_3_acc_5: 0.3880 - dense_3_acc_6: 0.0697 - dense_3_acc_7: 0.9111 - dense_3_acc_8: 0.2252 - dense_3_acc_9: 0.091 - ETA: 2s - loss: 16.1781 - dense_3_loss: 2.6096 - dense_3_acc: 0.5454 - dense_3_acc_1: 0.7480 - dense_3_acc_2: 0.3397 - dense_3_acc_3: 0.0711 - dense_3_acc_4: 0.8937 - dense_3_acc_5: 0.3928 - dense_3_acc_6: 0.0716 - dense_3_acc_7: 0.9121 - dense_3_acc_8: 0.2283 - dense_3_acc_9: 0.092 - ETA: 1s - loss: 16.1035 - dense_3_loss: 2.6051 - dense_3_acc: 0.5502 - dense_3_acc_1: 0.7507 - 

...

10000/10000 [==============================] - ETA: 1s - loss: 0.0050 - dense_3_loss: 5.8479e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 1s - loss: 0.0050 - dense_3_loss: 5.8447e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 1s - loss: 0.0050 - dense_3_loss: 5.8685e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 0s - loss: 0.0050 - dense_3_loss: 5.8753e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 

 2900/10000 [=======>......................] - ETA: 10s - loss: 0.0052 - dense_3_loss: 7.2276e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.000 - ETA: 11s - loss: 0.0054 - dense_3_loss: 6.7778e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.000 - ETA: 11s - loss: 0.0051 - dense_3_loss: 6.1484e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.000 - ETA: 10s - loss: 0.0050 - dense_3_loss: 5.9055e-04 - dense_3_acc: 1.0000 - dense_3_

 5800/10000 [================>.............] - ETA: 7s - loss: 0.0048 - dense_3_loss: 5.7258e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 7s - loss: 0.0048 - dense_3_loss: 5.7017e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 7s - loss: 0.0048 - dense_3_loss: 5.6770e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 7s - loss: 0.0047 - dense_3_loss: 5.6445e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 

 8700/10000 [=========================>....] - ETA: 4s - loss: 0.0047 - dense_3_loss: 5.5056e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 4s - loss: 0.0047 - dense_3_loss: 5.4937e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 4s - loss: 0.0047 - dense_3_loss: 5.4925e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 4s - loss: 0.0047 - dense_3_loss: 5.5044e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 

10000/10000 [==============================] - ETA: 1s - loss: 0.0047 - dense_3_loss: 5.4329e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 1s - loss: 0.0047 - dense_3_loss: 5.4416e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 1s - loss: 0.0047 - dense_3_loss: 5.4713e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 1.0000 - dense_3_acc_2: 1.0000 - dense_3_acc_3: 1.0000 - dense_3_acc_4: 1.0000 - dense_3_acc_5: 1.0000 - dense_3_acc_6: 1.0000 - dense_3_acc_7: 1.0000 - dense_3_acc_8: 1.0000 - dense_3_acc_9: 1.00 - ETA: 0s - loss: 0.0047 - dense_3_loss: 5.4573e-04 - dense_3_acc: 1.0000 - dense_3_acc_1: 

In [15]:
EXAMPLES = ['3 May 1979', '5 April 09', '21th of August 2016', 'Tue 10 Jul 2007', 'Saturday May 9 2018', 'March 3 2001', 'March 3rd 2001', '1 March 2001']
for example in EXAMPLES:
    
    source = string_to_int(example, Tx, human_vocab)
    print(source)
    source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source))).swapaxes(0,1)
    source=source.T
    source=source.reshape((1,source.shape[0],source.shape[1]))
    prediction = model.predict([source, s0, c0])
    prediction = np.argmax(prediction, axis = -1)
    output = [inv_machine_vocab[int(i)] for i in prediction]
    
    print("source:", example)
    print("output:", ''.join(output))

[6, 0, 24, 13, 34, 0, 4, 12, 10, 12, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]
source: 3 May 1979
output: 1979-05-03
[8, 0, 13, 27, 28, 21, 23, 0, 3, 12, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]
source: 5 April 09
output: 2009-04-05
[5, 4, 30, 20, 0, 26, 18, 0, 13, 31, 19, 31, 29, 30, 0, 5, 3, 4, 9, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]
source: 21th of August 2016
output: 2016-08-21
[30, 31, 17, 0, 4, 3, 0, 22, 31, 23, 0, 5, 3, 3, 10, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]
source: Tue 10 Jul 2007
output: 2007-07-10
[29, 13, 30, 31, 28, 16, 13, 34, 0, 24, 13, 34, 0, 12, 0, 5, 3, 4, 11, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]
source: Saturday May 9 2018
output: 2018-05-09
[24, 13, 28, 15, 20, 0, 6, 0, 5, 3, 3, 4, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]
source: March 3 2001
output: 2001-03-03
[24, 13, 28, 15, 20, 0, 6, 28, 16, 0, 5, 3, 3, 4, 36, 36, 3

In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 37)       0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 64)           0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 64)       17920       input_1[0][0]                    
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 30, 64)       0           s0[0][0]                         
                                                                 lstm_1[0][0]                     
          

                                                                 attention_weights[4][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[5][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[6][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[7][0]          
                                                                 bidirectional_1[0][0]            
                                                                 attention_weights[8][0]          
                                                                 bidirectional_1[0][0]            
          